In [187]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from crop_image import *
import imutils
from imutils import contours as imcnts
import skimage.io as io
%matplotlib inline

In [188]:
image = cv2.imread('omr.png')
cv2.imshow('omr.png',image)
cv2.waitKey(0)

-1

In [189]:
gray_scale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow('gray scale image.jpg',gray_scale)
cv2.waitKey(0)

-1

In [190]:
"""gaussian = cv2.GaussianBlur(image,(3,3),0)
cv2.imshow('image',gaussian)
cv2.waitKey(0)"""

"gaussian = cv2.GaussianBlur(image,(3,3),0)\ncv2.imshow('image',gaussian)\ncv2.waitKey(0)"

In [191]:
edged = cv2.Canny(gray_scale,50,200)
# cv2.imshow('edged.jpg',edged)
# cv2.waitKey(0)

In [192]:
contours,_= cv2.findContours(edged, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print("Number of Contours found = " + str(len(contours)))
  
cv2.drawContours(image, contours, -1,(0,255,0), 3)
  
# cv2.imshow('Contours.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

Number of Contours found = 4


In [193]:
#sorting all contours
if(len(contours)>0):
    contours=sorted(contours,key=cv2.contourArea,reverse=True)

In [194]:
cv2.drawContours(image, contours, 0,(0,255,0), 3)
  
# cv2.imshow('Contours3.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

In [195]:
paper_contour=None
for cnt in contours:
    peri = cv2.arcLength(cnt,True)
    print(peri)
    # to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
    ratio=0.02
    approx=cv2.approxPolyDP(cnt,ratio*peri,True)
    cv2.drawContours(image, [approx], 0,(0,0,255), 3)
    print(len(approx))
    coordinates=approx.ravel()
    for i in range(0,len(coordinates),2):
        x=coordinates[i]
        y=coordinates[i+1]
        cv2.putText(image, "vertex"+str(i//2), (x, y),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0)) 
    if(len(approx)==4):
        paper_contour=approx
        break
# cv2.imshow('Contours2.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

1906.0
4


In [196]:
print(paper_contour.shape)

(4, 1, 2)


In [197]:
paper= four_point_transform(image, paper_contour.reshape(4, 2))  
cv2.imshow('paper', paper)
cv2.waitKey(0)

-1

In [198]:
gray_scale_paper= four_point_transform(gray_scale, paper_contour.reshape(4, 2))  
# cv2.imshow('paper', gray_scale_paper)
# cv2.waitKey(0)

In [199]:
# _,thresholded=cv2.threshold(gray_scale_paper,230,255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
# cv2.imshow('paper1', thresholded)
# cv2.waitKey(0)

In [200]:
_,thresholded=cv2.threshold(gray_scale_paper,230,255, cv2.THRESH_BINARY_INV)
cv2.imshow('paper2', thresholded)
cv2.waitKey(0)

-1

In [201]:
pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [202]:
def test_cnts(cnts):
    cv2.drawContours(thresholded, cnts, -1,255, 3)
  
    cv2.imshow('Contours.jpg', thresholded)
    cv2.waitKey(0)

In [203]:
pre_question_cnts=[]
value=(5,50)
i=0
for cnt in pap_cnts:
    (x,y,w,h)=cv2.boundingRect(cnt)
    aspect_ratio=w/h
    if(w>=value[0] and w<=value[1] and h>=value[0] and h<=value[1] and aspect_ratio>=0.9 and aspect_ratio<=1.1):
        pre_question_cnts.append(cnt)

In [204]:
print(len(pre_question_cnts),len(pap_cnts))

300 369


In [224]:
if(len(pre_question_cnts)!=0):
    question_cnts,_=imcnts.sort_contours(pre_question_cnts,method='top-to-bottom')
    xs_set=set()
    i=0

    for cnt in question_cnts:
        (x,y,w,h)=cv2.boundingRect(cnt)
        if(all([ x-i not in xs_set and x+i not in xs_set for i in np.arange(0,5)])):
            xs_set.add(x)
    number_of_bubbles=len(xs_set)
    print(xs_set)
    print("number of bubbles : "+str(number_of_bubbles))
    xs=np.array(list(sorted(xs_set)))
    dist=np.append(xs[1:],xs[-1])-xs
    print("differences:" ,dist)
    distance_set=[]
    number_of_columns=1
    for diff in dist:
        if(len(distance_set)==0 or all([ diff-i != distance_set[-1] and diff+i != distance_set[-1] for i in np.arange(0,5)])):
            distance_set.append(diff)
    print(len(distance_set))
    distance_set.pop()
    print(distance_set)
    if(len(distance_set)>=3):
        for i in range(1,len(distance_set)-1):
            if(distance_set[i]>distance_set[i-1] and distance_set[i]>distance_set[i+1]):
                number_of_columns=number_of_columns+1
    number_of_choices=number_of_bubbles//number_of_columns
    print(number_of_choices,number_of_columns)

{193, 225, 295, 328, 74, 139, 107, 241, 177, 209, 311, 344, 90, 123, 279}
number of bubbles : 15
differences: [16 17 16 16 38 16 16 16 16 38 16 16 17 16  0]
6
[16, 38, 16, 38, 16]
5 3


In [206]:
tt=[i>2 for i in np.arange(1,5)]
print(all(tt))

False


In [207]:
# Answer_key= {0:1, 1:4, 2:0, 3:2}
# correct=0
# flag=False
# cv2.drawContours(paper,question_cnts,0,(255,0,0), 1)
# cv2.imshow('hgsvrgvj',paper)
# cv2.waitKey(0)
# cnt=question_cnts[0]
# peri = cv2.arcLength(cnt,True)
# #print(peri)
# # to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
# ratio=0.00005
# approx=cv2.approxPolyDP(cnt,ratio*peri,True)
# (x,y,w,h)=cv2.boundingRect(cnt)
# aspect_ratio=w/h
# print(len(approx))
# paper= four_point_transform(image, paper_contour.reshape(4, 2))  

In [208]:
print(len(question_cnts))

300


In [209]:
#test_cnts(question_cnts)

In [210]:
# _,thresholded=cv2.threshold(gray_scale_paper,150,255, cv2.THRESH_BINARY_INV)
# pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [211]:
colors=[(255,0,0)
,(0,255,0)
,(0,0,255)
,(0,255,255)
,(255,255,0)
,(0,0,255)
,(0,255,255)
,(255,0,255)
,(192,192,192)
,(128,128,128)
,(128,0,0)
,(128,128,0)
,(0,128,0)
,(128,0,128)
,(0,128,128)
,(0,0,128),
       (255,215,0)]

In [212]:
counter=0
qs=0
for (row,i) in enumerate(np.arange(0,len(question_cnts),number_of_columns*number_of_choices)):

    curr_row_cnts,_=imcnts.sort_contours(question_cnts[i:i+number_of_columns*number_of_choices])
    for k in np.arange(0,len(curr_row_cnts),number_of_choices):
        curr_ques_cnts=curr_row_cnts[k:k+number_of_choices]
        color1=colors[qs%len(colors)]
        cv2.drawContours(paper,curr_ques_cnts,-1,color1, 1)
        bubbled=None
        for (j,c) in enumerate(curr_ques_cnts):
            mask = np.zeros(thresholded.shape, dtype="uint8")
            #negative one at the last arguement to fill the contours
            cv2.drawContours(mask, [c], -1, 255, -1)
            mask= cv2.bitwise_and(thresholded, mask)
            total= cv2.countNonZero(mask)

            if bubbled is None or total >bubbled[0]:
                bubbled= (total, j)
            color= (0, 0, 255)
        print(f"qestion#{qs+1}: {chr(bubbled[1]+ord('A'))}")
        qs+=1
    counter+=1
print(counter)
counter=0
cv2.imshow('paper', paper)
cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

qestion#1: E
qestion#2: C
qestion#3: F
qestion#4: C
qestion#5: G
qestion#6: C
qestion#7: C
qestion#8: C
qestion#9: D
qestion#10: C
qestion#11: E
qestion#12: B
qestion#13: D
qestion#14: A
qestion#15: G
qestion#16: D
qestion#17: B
qestion#18: F
qestion#19: F
qestion#20: D
qestion#21: B
qestion#22: F
qestion#23: A
qestion#24: F
qestion#25: A
qestion#26: F
qestion#27: A
qestion#28: G
qestion#29: A
qestion#30: A
qestion#31: G
qestion#32: C
qestion#33: G
qestion#34: D
qestion#35: G
qestion#36: E
qestion#37: B
qestion#38: F
qestion#39: G
qestion#40: G
qestion#41: G
qestion#42: C
qestion#43: B
22


In [213]:
for i,cnt in enumerate(question_cnts):
    color1=(255,0,0) if i%2==0 else (0,0,255)
    cv2.drawContours(paper,[cnt],-1,color1, 1)
# cv2.imshow('paper', paper)
# cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

In [214]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()



In [215]:
print(contours)

[array([[[24, 13]],

       [[24, 14]],

       [[24, 15]],

       ...,

       [[27, 13]],

       [[26, 13]],

       [[25, 13]]], dtype=int32), array([[[ 24, 566]],

       [[ 25, 566]],

       [[ 26, 566]],

       ...,

       [[ 27, 566]],

       [[ 26, 566]],

       [[ 25, 566]]], dtype=int32), array([[[428,  13]],

       [[428,  14]],

       [[428,  15]],

       ...,

       [[428,  16]],

       [[428,  15]],

       [[428,  14]]], dtype=int32), array([[[23, 11]],

       [[22, 12]],

       [[22, 13]],

       ...,

       [[26, 11]],

       [[25, 11]],

       [[24, 11]]], dtype=int32)]


In [216]:
f"adham{chr(bubbled[1]+ord('A'))}"

'adhamB'